# asyncio 基本概念

asyncio 运算的核心就是EventLoop，它是一个指挥中心，面对着很多任务，负责决定执行哪个任务(task)，同时执行的任务只能有一个。注意EventLoop的管理对象是task（可能具有自己的数据结构）。

**coroutine 的两个重要概念：coroutine object、coroutine function** 运行coroutine function将返回coroutine object

In [ ]:
import asyncio
# 以 async def 开头定义的函数为coroutine function,该调用该函数的返回将是一个coroutine object
async def main():
    print("hello")
    await asyncio.sleep(1)
    print("welch !")
coro = main()
# Execute the coroutine and return the result.
asyncio.run(coro)

## 例子

In [ ]:
import asyncio
import time

async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
   print(f"start at {time.time()}")
   await say_after(1, "Bob")
   await say_after(2, "welch")
   print(f"end at {time.time()}")

asyncio.run(main())

函数执行流程如下（猜想）：
1.调用main()函数，并返回一个coroutine；
2.asyncio.run将上一步的coroutine变成一个任务执行注册到eventloop中，由于此时只有一个任务，则执行main()
3.遇到一个await coroutine object（await say_after()）将coroutine object作为一个task注册到eventloop中，并告诉eventloop该函数需要等待io，将控制权交给eventloop。此时，eventloop中存在两个task：main、say_after，同时mian、say_after存在一个依赖关系，mian需要等待say_after执行完毕才能执行，所以eventloop执行say_after这个任务。进入say_after这个函数后又遇到了await coroutine object （await asyncio.sleep(delay)），此时，eventloop中又多了一个task，他们的依赖关系为:main-->say_after-->sleep。然后eventloop通过sleep、say_after、main的顺序执行到main函数，然后需要下一个await coroutine object。
4. eventloop根据任务的依赖关系决定选择哪个任务执行，直到所有的任务都执行完毕。

注意：上述过程eventloop并没有主动的选择哪个任务执行，它并没有控制权。而是task主动将控制权交给eventloop（函数执行完毕或者出现await），eventloop根据当前任务依赖关系选择执行。

我们发现在main函数中

In [ ]:
await say_after(1, "Bob")
await say_after(2, "welch")

这两行代码是顺序执行（同步），执行需要三秒钟。为了解决这个问题，asyncio提供了其他方法并发执行。

In [ ]:
async def main():
   task1 = asyncio.create_task(say_after(1, "Bob"))
   task2 = asyncio.create_task(say_after(2, "welch"))
   print(f"start at {time.time()}")
   await task1
   await task2
   print(f"end at {time.time()}")

此时函数的执行只需要2秒，与上一个版本不同，上述代码首先利用asyncio.create_task将所有的coroutine变成task，然后利用await task （await task1、await task2）告诉eventloop这里有两个任务需要完成。实现协程的并发执行，程序执行如下：
1. main task取得控制权，注册task1、task2到eventloop
2. await task1 将控制权交给 say_after(1, "Bob")（await task 省略了将coroutine注册为任务的过程），并遇到另一个await coroutine，此时控制权交给了eventloop
3. eventloop发现存在三个任务（main、task1，task2）选择了其他任务执行。从而减少了

# 1. yield

字典为动词“to yield”给出了两个释义：产出和让步。对于 Python 生成器
中的 yield 来说，这两个含义都成立。yield item 这行代码会产出一
个值，提供给 next(...) 的调用方；此外，还会作出让步，暂停执行
生成器，让调用方继续工作，直到需要使用另一个值时再调用
next()。调用方会从生成器中拉取值。

In [ ]:
def simple_coroutine(): # ➊
    print('-> coroutine started')
    x = yield # ➋
    print('-> coroutine received:', x)
mycoro = simple_coroutine()
mycoro

In [ ]:
next(mycoro)
next(mycoro)

In [ ]:
mycoro1 = simple_coroutine()

In [ ]:
next(mycoro1)

In [ ]:
mycoro1.send(10)

In [ ]:
import asyncio
async def hello():
    print("Hello world!")
    r = await asyncio.sleep(1)
    print("Hello again!")